**Reference Link:** [Building AI Agents with LangChain (Analytics Vidhya)](https://courses.analyticsvidhya.com/courses/take/building-ai-agents-with-langchain/lessons/62141622-introduction-to-agents-and-tools)

# Build a Multi-User Conversational Tool-Calling Agentic AI Research Assistant with LangChain

This demo will cover building AI Agents with the legacy LangChain `AgentExecutor`. These are fine for getting started, but for working with more advanced agents, LangChain recommends to use LangGraph, which we will cover in the future demos.

Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to stop.

Here we will build a multi-user Conversational Agent which can refer to previous conversations and give more contextual answers by storing agent messages to a SQL database

![](https://i.imgur.com/lHWqaT9.png)



## Install OpenAI, and LangChain dependencies

In [0]:
!pip install -q langchain==0.3.14
!pip install -q langchain-openai==0.3.0
!pip install -q langchain-community==0.3.14

In [0]:
!pip install -q markitdown

## Enter Open AI API Key

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [0]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

Enter Tavily Search API Key: ··········


## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [0]:
WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

Enter WeatherAPI API Key: ··········


## Setup Environment Variables

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Web Search tool with information extraction
- Weather tool

![](https://i.imgur.com/TyPAYXE.png)

In [0]:
from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
import json
from warnings import filterwarnings
filterwarnings('ignore')

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links."""
    print('Calling web search tool')
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract content from a URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content

    with ThreadPoolExecutor() as executor:
        for result in tqdm(results):
            try:
                future = executor.submit(extract_content, result['url'])
                # Wait for up to 60 seconds for the task to complete
                content = future.result(timeout=60)
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")

    return docs


@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather of the queried location."""
    print('Calling weather tool')
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [0]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.

                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Act as a helpful assistant.\n                You run in a loop of Thought, Action, PAUSE, Observation.\n                At the end of the loop, you output an Answer.\n                Use Thought to describe your thoughts about the question you have been asked.\n                Use Action to run one of the actions available to you - then return PAUSE.\n                Observation will be the result of running those actions.\n                Repeat till you get to the answer for the given user query.\n\n                Use the following workflow format:\n                  Question: the input task you must solve\n                  Thought: you should always think about what to do\n                  Action: the action to take which can be any of the following:\n                            - break it into smaller steps if needed\n                            - see if you can

Now, we can initalize the agent with the LLM, the prompt, and the tools.

The agent is responsible for taking in input and deciding what actions to take.

REMEMBER the Agent does not execute those actions - that is done by the AgentExecutor

Note that we are passing in the model `chatgpt`, not `chatgpt_with_tools`.

That is because `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

This should ideally be used with an LLM which supports tool \ function calling

In [0]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

Finally, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [0]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               early_stopping_method='force',
                               max_iterations=10)

In [0]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = chatgpt.invoke(query)
response.content

"As of my last update, Nvidia's Q4 2024 earnings call has not occurred. However, I can provide a general outline of what is typically discussed during such earnings calls. When Nvidia holds its Q4 2024 earnings call, you can expect the following key points to be covered:\n\n1. **Financial Performance**: Discussion of revenue, net income, earnings per share, and other key financial metrics compared to previous quarters and the same quarter in the prior year.\n\n2. **Segment Performance**: Insights into the performance of different business segments, such as gaming, data center, professional visualization, and automotive.\n\n3. **Market Trends**: Commentary on market conditions affecting Nvidia's business, including demand for GPUs, AI, and other technologies.\n\n4. **Product Updates**: Information on recent product launches, updates, and future product roadmaps.\n\n5. **Strategic Initiatives**: Discussion of strategic priorities, including partnerships, acquisitions, and investments in 

In [0]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})

Calling web search tool


 20%|██        | 1/5 [00:01<00:07,  1.98s/it]

Error extracting from url: https://thetranscript.net/transcript/5857/nvidia-q4-2024-earnings-call-transcript - 'NoneType' object has no attribute 'strip'


 80%|████████  | 4/5 [00:05<00:01,  1.45s/it]

Error extracting from url: https://www.earningscall.ai/stock/analyze/NVDA-2024-Q4 - 'NoneType' object has no attribute 'strip'


100%|██████████| 5/5 [00:06<00:00,  1.37s/it]


In [0]:
response

{'query': "Summarize the key points discussed in Nvidia's Q4 2024 earnings call",
 'output': "In Nvidia's Q4 2024 earnings call, several key points were highlighted:\n\n1. **Record Financial Performance**: Nvidia reported a record quarterly revenue of $22.1 billion, a 22% increase from the previous quarter and a 265% increase year-over-year. The full-year revenue reached $60.9 billion, up 126% from the previous year.\n\n2. **Data Center Growth**: The data center segment achieved a record $18.4 billion in revenue for the quarter, marking a 27% rise from Q3 and a 409% year-over-year growth. This growth was driven by the demand for Nvidia's Hopper GPU computing platform and InfiniBand networking.\n\n3. **AI and Accelerated Computing**: Nvidia emphasized the transition from general-purpose to accelerated computing, driven by the demand for AI and generative AI applications. The company is focusing on AI factories and AI infrastructure to support this shift.\n\n4. **Gross Margin and Profita

In [0]:
from IPython.display import display, Markdown

display(Markdown(response['output']))

In Nvidia's Q4 2024 earnings call, several key points were highlighted:

1. **Record Financial Performance**: Nvidia reported a record quarterly revenue of $22.1 billion, a 22% increase from the previous quarter and a 265% increase year-over-year. The full-year revenue reached $60.9 billion, up 126% from the previous year.

2. **Data Center Growth**: The data center segment achieved a record $18.4 billion in revenue for the quarter, marking a 27% rise from Q3 and a 409% year-over-year growth. This growth was driven by the demand for Nvidia's Hopper GPU computing platform and InfiniBand networking.

3. **AI and Accelerated Computing**: Nvidia emphasized the transition from general-purpose to accelerated computing, driven by the demand for AI and generative AI applications. The company is focusing on AI factories and AI infrastructure to support this shift.

4. **Gross Margin and Profitability**: The gross margin improved to 76.0% in Q4, up from 74.0% in Q3 and 63.3% the previous year. Net income for Q4 was $12.3 billion, a 33% increase from Q3 and a 769% increase year-over-year.

5. **Gaming and Other Segments**: Gaming revenue was $2.87 billion, up 56% year-over-year. Nvidia also reported growth in its Pro Visualization and Automotive segments.

6. **Future Outlook**: Nvidia anticipates Q1 fiscal 2025 revenue to be around $24 billion, with continued growth in data center and Pro Visualization segments. The company expects gross margins to remain strong.

7. **Challenges and Innovations**: Nvidia acknowledged the challenges in the semiconductor industry, including rapid technological advancements and competition. The company is focusing on innovation and new product cycles to maintain its market position.

Overall, Nvidia's Q4 2024 earnings call highlighted the company's strong financial performance, driven by its leadership in AI and accelerated computing, and its strategic focus on innovation and growth across various sectors.

In [0]:
query = """Summarize the key points discussed in Intel's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling web search tool


100%|██████████| 5/5 [00:03<00:00,  1.44it/s]


In Intel's Q4 2024 earnings call, the company reported a profit of $2.7 billion, compared to a loss of $0.7 billion in the same period the previous year. The earnings per share (EPS) were $0.63, up from a loss of $0.16 per share last year. Excluding certain items, adjusted earnings were $2.3 billion or $0.54 per share, surpassing analysts' expectations of $0.45 per share. Revenue for the quarter was $15.4 billion, up from $14.0 billion the previous year. For the next quarter, Intel provided guidance for EPS of $0.13 and revenue between $12.2 billion and $13.2 billion.

In [0]:
query = """which company's future outlook looks to be better?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

To provide an accurate assessment of a company's future outlook, I would need to know which specific companies you are interested in comparing. Please provide the names of the companies you would like to evaluate.

The agent is doing pretty well but unfortunately it doesn't remember conversations. We will use some user-session based memory to store this and dive deeper into this in the next video.

## Build and Test Multi-User Conversational AI Agent

We will now use `SQLChatMessageHistory` which we learnt in the previous module to store separate conversation histories per user or session.

This will help us build a conversational Agentic Chatbot which will be accessed by many users at the same time

In [0]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove ALL conversation histories
# ok if you get rm: cannot remove 'memory.db': No such file or directory  because initially no memory exists
!rm memory.db

rm: cannot remove 'memory.db': No such file or directory


In [0]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

# create a conversation chain + agent which can load memory based on specific user or session id
agentic_chatbot = RunnableWithMessageHistory(
    agent_executor,
    get_session_history_db,
    input_messages_key="query",
    history_messages_key="history",
)

# function to call the agent show results per user session
from IPython.display import display, Markdown
def chat_with_agent(prompt: str, session_id: str):
    response = agentic_chatbot.invoke({"query": prompt},
                                      {'configurable': { 'session_id': session_id}})
    display(Markdown(response['output']))

Let's now simulate User 1 using the agent

In [0]:
user_id = 'john001'
prompt = "Summarize the key points discussed in Nvidia's Q4 2024 earnings call"
chat_with_agent(prompt, user_id)

/usr/local/lib/python3.11/dist-packages/langchain_core/runnables/history.py:608: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


Calling web search tool


 20%|██        | 1/5 [00:00<00:03,  1.13it/s]

Error extracting from url: https://www.earningscall.ai/stock/analyze/NVDA-2024-Q4 - 'NoneType' object has no attribute 'strip'


 40%|████      | 2/5 [00:01<00:02,  1.15it/s]

...

 80%|████████  | 4/5 [00:05<00:01,  1.46s/it]

Error extracting from url: https://www.investiment.io/symbols/NVDA/earnings/transcripts/2024/4 - 'NoneType' object has no attribute 'strip'


100%|██████████| 5/5 [00:05<00:00,  1.06s/it]

Error extracting from url: https://investor.nvidia.com/news/press-release-details/2024/NVIDIA-Announces-Financial-Results-for-Fourth-Quarter-and-Fiscal-2024/ - 403 Client Error: Forbidden for url: https://investor.nvidia.com/news/press-release-details/2024/NVIDIA-Announces-Financial-Results-for-Fourth-Quarter-and-Fiscal-2024/


Nvidia's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Nvidia reported a record revenue of $22.1 billion for Q4, up 22% sequentially and 265% year-over-year. The fiscal year 2024 revenue was $60.9 billion, up 126% from the previous year.

2. **Data Center Growth**: Data center revenue reached $18.4 billion in Q4, driven by the NVIDIA Hopper GPU computing platform and InfiniBand networking. The demand for AI infrastructure, particularly for generative AI and large language models, was a significant growth driver.

3. **AI and Generative AI**: Nvidia emphasized the growing adoption of AI across industries, with significant contributions from AI inference. The company is collaborating with major tech firms like Google and Microsoft to enhance AI capabilities.

4. **Gaming Sector**: Gaming revenue was $2.87 billion, up 56% year-over-year. Nvidia launched the GeForce RTX 40 Super Series GPUs, which saw strong consumer demand.

5. **Automotive and Other Sectors**: Automotive revenue was $281 million, with Nvidia DRIVE platform adoption continuing to grow. The company is also seeing increased demand in healthcare and financial services for AI applications.

6. **Geographical Performance**: While growth was strong globally, data center revenue in China declined due to U.S. export control regulations. Nvidia is shipping alternative products to China that do not require a license.

7. **Future Outlook**: Nvidia expects Q1 2025 revenue to be around $24 billion, with continued growth in data center and professional visualization sectors.

Overall, Nvidia's earnings call underscored its strong financial performance, driven by significant growth in AI and data center sectors, and its strategic collaborations to enhance AI capabilities across various industries.

In [0]:
prompt = "What about Intel?"
chat_with_agent(prompt, user_id)

Calling web search tool


100%|██████████| 5/5 [00:03<00:00,  1.46it/s]


Intel's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Intel reported a profit of $2.67 billion for Q4, compared to a loss of $0.7 billion in the same period last year. Earnings per share (EPS) were $0.63, up from a loss of $0.16 per share last year. Adjusted earnings were $2.3 billion or $0.54 per share, exceeding analysts' expectations of $0.45 per share.

2. **Revenue**: The company posted revenue of $15.4 billion for Q4, up from $14.0 billion in the same period last year, surpassing the expected $15.14 billion.

3. **Guidance**: For the next quarter, Intel provided guidance for EPS of $0.13 and revenue between $12.2 billion and $13.2 billion.

4. **Market Conditions**: Intel's performance was bolstered by improvements in its data center and client computing groups, despite ongoing challenges in the PC market.

5. **Strategic Initiatives**: Intel continues to focus on its IDM 2.0 strategy, which includes expanding its manufacturing capabilities and investing in new technologies to drive future growth.

Overall, Intel's Q4 2024 earnings call reflected a strong recovery from the previous year's losses, with better-than-expected financial results and a positive outlook for the upcoming quarter.

In [0]:
prompt = "Which company seems to be doing better?"
chat_with_agent(prompt, user_id)

Based on the Q4 2024 earnings calls for both Nvidia and Intel, Nvidia appears to be performing better overall. Here are some key comparisons:

1. **Revenue Growth**: Nvidia reported a record revenue of $22.1 billion for Q4, with a significant year-over-year growth of 265%. In contrast, Intel's revenue for Q4 was $15.4 billion, with a more modest increase from the previous year.

2. **Profitability**: Nvidia's strong financial performance is highlighted by its substantial revenue growth and profitability driven by its data center and AI sectors. Intel, while profitable, showed a recovery from a previous loss, indicating a turnaround rather than explosive growth.

3. **Market Drivers**: Nvidia's growth is largely driven by the booming demand for AI infrastructure and generative AI, areas where it has a strong market position. Intel's growth, while positive, is more focused on recovering from past challenges and stabilizing its core businesses.

4. **Future Outlook**: Nvidia's guidance for the next quarter suggests continued strong growth, particularly in data centers and AI. Intel's guidance, while positive, indicates a more cautious outlook with lower expected revenue compared to Nvidia.

Overall, Nvidia's performance and growth prospects, particularly in AI and data centers, position it as the stronger performer compared to Intel in this period.

Let's now simulate User 2 using the agent

In [0]:
user_id = 'bond007'
prompt = "how is the weather in Bangalore today? Show detailed statistics"
chat_with_agent(prompt, user_id)

Calling weather tool


The weather in Bangalore today is as follows:

- **Condition**: Mist
- **Temperature**: 18.3°C (64.9°F)
- **Feels Like**: 18.3°C (64.9°F)
- **Wind**: 6.7 mph (10.8 kph) from the East-Southeast (ESE)
- **Wind Gusts**: Up to 10.6 mph (17.0 kph)
- **Humidity**: 94%
- **Cloud Cover**: 50%
- **Pressure**: 1016.0 mb (30.0 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Visibility**: 2.0 km (1.0 mile)
- **UV Index**: 0.4
- **Dew Point**: 15.3°C (59.5°F)

The weather is currently misty with moderate humidity and low visibility.

In [0]:
user_id = 'bond007'
prompt = "what about Dubai?"
chat_with_agent(prompt, user_id)

Calling weather tool


The weather in Dubai today is as follows:

- **Condition**: Clear
- **Temperature**: 16.2°C (61.2°F)
- **Feels Like**: 16.2°C (61.2°F)
- **Wind**: 12.3 mph (19.8 kph) from the East (E)
- **Wind Gusts**: Up to 21.9 mph (35.2 kph)
- **Humidity**: 45%
- **Cloud Cover**: 0% (Clear skies)
- **Pressure**: 1021.0 mb (30.15 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Visibility**: 6.0 km (3.0 miles)
- **UV Index**: 0.0
- **Dew Point**: 5.0°C (41.1°F)

The weather is clear with moderate wind and low humidity.

In [0]:
user_id = 'bond007'
prompt = "which city is hotter?"
chat_with_agent(prompt, user_id)

Dubai is currently hotter than Bangalore. The temperature in Dubai is 16.2°C (61.2°F), while in Bangalore, it is 18.3°C (64.9°F).